# 2 - Algorithm Design

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook,tnrange,tqdm_pandas,tqdm
tqdm.pandas(tqdm())

import os
import cPickle as pickle

import datetime as dt

import ipywidgets as widgets


import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
#notify if cell is finished for cells taking longer than 30 seconds
%autonotify -a 30

0it [00:00, ?it/s]

<IPython.core.display.Javascript object>

### Path to your urinal-data-28-nov_clean _pickle_ file

In [2]:
# pickle_name = "urinal-data-28-nov_clean.p"
# pickle_path = "F:\\Research\\ben\\grideye_urinal"
# pickle_full = os.path.join(pickle_path, pickle_name)

# print("looking in", pickle_full)
pickle_full = "urinal-data-28-nov_clean.p"

### Load Data

In [3]:
#---------- read in pickle----------   
if os.path.isfile(pickle_full):
    print("loading pickle")
    df = pd.read_pickle(pickle_full)
   
else:
    print("Did you run 1- Raw Data Visualisation?")

loading pickle


we want indexes not, timestamps, as our time data is only recorded to the nearest second (yet we have a sampling rate of 10 frames/second)

In [4]:
df.reset_index(inplace=True)

In [5]:
df.sample(2)

,time,P0,P1,P2,P3,P4,P5,P6,P7,P8,...,P54,P55,P56,P57,P58,P59,P60,P61,P62,P63
641008,2017-11-24 13:10:22,23.0,22.25,22.0,21.5,21.75,22.50,22.75,23.00,22.0,...,21.25,22.75,21.75,22.75,24.00,22.75,23.50,24.25,21.5,23.50
93419,2017-11-23 19:56:49,23.0,24.50,23.5,24.0,23.50,23.25,23.75,21.75,23.5,...,24.25,22.25,22.25,24.50,23.25,22.75,24.75,24.00,23.0,23.25


## Adaptive Threshold
In _1- Raw Data Visualisation_ we saw how noisy our data is. 
With some additional testing, it's pretty clear simply looking for pixels above a set temperature is not sufficient, the ambient air temperature just varies too [much](https://www.sciencedirect.com/science/article/pii/S0360132396000340)!

<img src="layout_sections.png" alt="grideye camera view" width="350" height="350" align="left">So: 
- We want to find a warm body in the of mess pixels. 
- We have a pretty good idea of where the warm body will appear. 
- We have an area where no person will be (the empty space)




These [researchers](http://www.scirp.org/jouRNAl/PaperInformation.aspx?PaperID=74726) achieved 97% accuracy finding a person with the Grid-EYE.  
By adapting their algorithm, we created an adaptive threshold algorithm for determining a urinal __"stay"__ which goes something like:

1. get background temperature (Empty Space) ($T_b$)
2. Calculate mean of highest three temperatures for each section  ($T_m$)
3. Compare $T_b$ to $T_m$ ($T_m - T_b > threshold$) and greater than an absolute temperature ($T_{abs}$) 
4. Determine a "__stay__" by thresholding the time

### Grab the Urinal Section

We first create a "mask" which contains a list of column names corresponding to the Urinal section  
In this case we ignore the top 2 rows, as no one in the office is above 7 foot!

In [6]:
height = 6
n = 8
width_left = 2
width_middle = 3
width_right = 3
bottom_trim = 0

left_mask   = []
middle_mask = []
right_mask  = []
for y in range(n-height,n-bottom_trim):
    # LEFT
    for x in range(width_left):
        left_mask.append(x+y*n) 
    # MIDDLE
    for x in range(width_middle):
         middle_mask.append(x+y*n+width_left) 
    # RIGHT
    for x in range(width_right):
         right_mask.append(x+y*n+width_left+width_middle) 

left_mask = ["P"+str(x) for x in left_mask]  
middle_mask = ["P"+str(x) for x in middle_mask]  
right_mask = ["P"+str(x) for x in right_mask]  


### Grab the background temperature ($T_b$)
Assuming no person (or hot object) will loiter near the roof,  
we take the background temperature values as the mean of top 2 rows (per section<sup>1</sup>)  
___
<sup>1</sup><sub>Per section, as there AC are vents above the left and right urinals and not the middle, therefore the temperature is generally higher in the middle.</sub>

In [7]:
height = 6 # top 2 rows
n = 8
width_left = 2
width_middle = 3
width_right = 3

left_top   = []
middle_top = []
right_top  = []
for y in range(0,n-height):
    # LEFT
    for x in range(width_left):
        left_top.append(x+y*n) 
    # MIDDLE
    for x in range(width_middle):
         middle_top.append(x+y*n+width_left) 
    # RIGHT
    for x in range(width_right):
         right_top.append(x+y*n+width_left+width_middle) 

left_top = ["P"+str(x) for x in left_top]  
middle_top = ["P"+str(x) for x in middle_top]  
right_top = ["P"+str(x) for x in right_top]  

df[middle_top].sample(10)

,P2,P3,P4,P10,P11,P12
1725008,22.75,21.75,23.25,23.00,22.75,23.00
1423259,23.25,23.00,22.75,23.75,23.25,22.75
491087,21.75,22.00,22.50,23.00,22.50,22.00
1728053,22.25,22.75,23.25,22.75,22.25,22.00
3379528,23.00,22.00,23.75,22.50,23.25,22.75
2499394,22.50,23.00,23.25,23.50,23.75,22.50
1457148,23.00,24.75,23.00,23.00,23.50,22.00
467994,21.75,22.00,22.50,22.25,22.50,21.75
2248858,23.00,22.00,23.00,24.00,23.75,23.50
334739,21.75,22.50,23.75,22.75,22.75,22.50


### Calculate the Background temperature ($T_b$)
We take the background temperature every minute, therefore making sure long-term changes in ambient temperature are considered 

In [8]:
window = 600 # 1 minute window for background temp

left_thresh = []
middle_thresh = []
right_thresh = []

for t in tnrange(window/2,len(df),window):  
    left_thresh.append(df.loc[t-window/2:t+window/2,:][left_top].mean().mean())
    middle_thresh.append(df.loc[t-window/2:t+window/2,:][middle_top].mean().mean())
    right_thresh.append(df.loc[t-window/2:t+window/2,:][right_top].mean().mean())



<IPython.core.display.Javascript object>

### Add minute to minute background temperatures ($T_b$) to dataframe
We also error check to test the columns are of equal length

In [9]:
left_background = [x for x in left_background for _ in range(window)]    
middle_background = [x for x in middle_background for _ in range(window)]    
right_background = [x for x in right_background for _ in range(window)]  

if len(left_background) > len(df):
    left_background = left_background[0:-(len(left_background) - len(df))]
    middle_background = middle_background[0:-(len(middle_background) - len(df))]
    right_background = right_background[0:-(len(right_background) - len(df))]
elif len(left_thresh) < len(df):    
    left_background.extend([left_background[-1]]*(len(df) - len(left_background)))
    middle_background.extend([middle_background[-1]]*(len(df) - len(middle_background)))
    right_background.extend([right_background[-1]]*(len(df) - len(right_background)))

if not (len(left_background) == len(middle_background) == len(right_background) == len(df)):
    raise AssertionError()

In [10]:
# pickle_name = "urinal-data-28-nov_clean.p"
# pickle_path = "F:\\Research\\ben\\grideye_urinal"
# pickle_full = os.path.join(pickle_path, pickle_name)

# print("looking in", pickle_full)
pickle_full = 'compare_temps.p'


### Calculate the mean of the highest 3 temperatures per frame ($T_m$)
We find the highest three because this is approximately the number of pixels a person occupies in a frame.  
If we look at the mean of all pixels, the value calculated is generally too close to the background temperature.  
___
<sub>I've created my own function here as I found it to be significantly faster than using numpy's _argpartition_ or bottleneck's _partition_</sub>


In [12]:

if os.path.isfile(pickle_full):
    print("loading pickle")
    compare_temps = pd.read_pickle(pickle_full)
    left = compare_temps[compare_temps["position"] == 'left']
    middle = compare_temps[compare_temps["position"] == 'middle']
    right = compare_temps[compare_temps["position"] == 'right']
else:
    print("creating pickle...")
    print("this will take some time...")
    
    left  = df[left_mask].copy()
    middle = df[middle_mask].copy()
    right = df[right_mask].copy()
     
    # Define a function which calculates
    # the highest three numbers from a list 
    # and calculate their mean
    # O(n)
    def top_three_mean(numbers):
        count = 0
        m1 = m2 = m3 = float('-inf')
        for number in numbers:
            count += 1
            if number >= m3:
                if number >= m2:
                    if number >= m1:
                        m1, m2, m3 = number, m1, m2   
                    else:
                        m3 = m2
                        m2 = number
                else:
                    m3 = number

        tri_mean = np.mean((m1,m2,m3))          
        return tri_mean if count >= 2 else None

    # Apply top_three_mean to each urinal section
    left["mean3"] = left.progress_apply(top_three_mean,axis = 1)
    middle["mean3"] = middle.progress_apply(top_three_mean,axis = 1)
    right["mean3"] = right.progress_apply(top_three_mean,axis = 1)
    
    # assign the previous calculated background temperatures to each urinal section
    left["background_temp"] = left_background
    middle["background_temp"] = middle_background
    right["background_temp"] = right_background
    
    # specify the urinal section (this is more convenient for pickling)
    left["position"] = 'left'
    middle["position"] = 'middle'
    right["position"] = 'right'
    
    compare_temps = pd.concat([left.iloc[:,-3:],middle.iloc[:,-3:],right.iloc[:,-3:]],axis = 0)
    
    compare_temps.to_pickle(pickle_full)
    
    left = compare_temps[compare_temps["position"] == 'left']
    middle = compare_temps[compare_temps["position"] == 'middle']
    right = compare_temps[compare_temps["position"] == 'right']
    

loading pickle


### Compare background temperature to highest three temperatures
$T_m - T_b > T_{th}$ __AND__ $T_m > T_{abs}$  
These two comparisons are converted to a boolean __1__ or __0__

$T_{Th}$ and $T_{abs}$ are determined from [Trofimova et. al.](http://www.scirp.org/jouRNAl/PaperInformation.aspx?PaperID=74726)

In [13]:
Tabs = 24.5
Tth = 1.5
df_temp_thresh = pd.DataFrame({"time":df["time"]})
df_temp_thresh["left"] = ((left["mean3"] - left["background_temp"] > Tth) & (left["mean3"] > Tabs)).astype(int)
df_temp_thresh["middle"] = ((middle["mean3"] - middle["background_temp"] > Tth) & (middle["mean3"] > Tabs)).astype(int)
df_temp_thresh["right"] = ((right["mean3"] - right["background_temp"] > Tth) & (right["mean3"] > Tabs)).astype(int)
df_temp_thresh.set_index("time",inplace=True)
df_temp_thresh.sample(5)

,left,middle,right
time,,,
2017-11-25 18:12:24,0,0,0
2017-11-24 05:21:08,0,0,0
2017-11-26 08:40:27,0,0,0
2017-11-24 22:58:08,0,0,0
2017-11-26 20:28:05,0,0,0


### Differentiate Binary values
Differentiating is a quick way of finding a state change,  
1 corresponds to a _walk in_, -1 corresponds to a _walk out_

In [18]:
df_temp_diff = df_temp_thresh.diff(axis=0)[1:]
df_temp_diff.reset_index(inplace=True)


### Remove all zeros
for the differentiated DataFrame zero corresponds to a state remaining the same.  
We only care about state changes!

In [23]:
df_temp_diff_redL =  pd.concat([df_temp_diff["left"][df_temp_diff["left"] != 0],df_temp_diff["time"][df_temp_diff["left"] != 0]],axis = 1)
df_temp_diff_redM = pd.concat([df_temp_diff["middle"][df_temp_diff["middle"] != 0],df_temp_diff["time"][df_temp_diff["middle"] != 0]],axis = 1)
df_temp_diff_redR = pd.concat([df_temp_diff["right"][df_temp_diff["right"] != 0],df_temp_diff["time"][df_temp_diff["right"] != 0]],axis = 1)

df_temp_diff_redL.columns = ["in_out","time"]
df_temp_diff_redM.columns = ["in_out","time"]
df_temp_diff_redR.columns = ["in_out","time"]

df_temp_diff_redR.head(5)

,in_out,time
130,1.0,2017-11-23 16:57:14
131,-1.0,2017-11-23 16:57:14
188,1.0,2017-11-23 16:57:21
189,-1.0,2017-11-23 16:57:21
225,1.0,2017-11-23 16:57:25


### Error Checking
Check that:
1. The data started with no person in frame
2. If a person walked in, they then walked out
3. The data ended with no person in frame

In [24]:
#check in_out always alternates-->it always does
prev = -1
test = df_temp_diff_redL
for i in tnrange(len(test)):
    
    curr = test["in_out"].iloc[i]
    if curr == prev:
        print(test["time"].iloc[i])
        raise AssertionError()
    prev = curr

    
assert df_temp_diff_redL["in_out"].sum() == 0 and df_temp_diff_redL["in_out"].iloc[-1] == -1 
assert df_temp_diff_redM["in_out"].sum() == 0 and df_temp_diff_redM["in_out"].iloc[-1] == -1
assert df_temp_diff_redR["in_out"].sum() == 0 and df_temp_diff_redR["in_out"].iloc[-1] == -1


### Convert 1 to "in" and -1 to "out" for readability

In [25]:
def in_out(b):
    try:
        if int(b) == 1:
            return "in"
        elif int(b) == -1:
            return "out"
        else:
            return b
    except:
        return b
    
df_temp_diff_redL["in_out"]    = df_temp_diff_redL["in_out"].apply(in_out)
df_temp_diff_redM["in_out"]    = df_temp_diff_redM["in_out"].apply(in_out)
df_temp_diff_redR["in_out"]    = df_temp_diff_redR["in_out"].apply(in_out)


### Annotate the position

In [26]:
df_temp_diff_redL["Position"] = ["left" for x in range(len(df_temp_diff_redL))]
df_temp_diff_redM["Position"] = ["middle" for x in range(len(df_temp_diff_redM))]
df_temp_diff_redR["Position"] = ["right" for x in range(len(df_temp_diff_redR))]

### Concatenate positions for time sorting

In [27]:
rsv = df_temp_diff_redL
rsv = rsv.append(df_temp_diff_redM)
rsv = rsv.append(df_temp_diff_redR)

rsv.sort_index(inplace=True)
rsv.sample(5)

,in_out,time,Position
1180727,in,2017-11-25 06:08:46,right
20009,out,2017-11-23 17:38:19,right
1729751,out,2017-11-25 23:24:33,right
1353411,out,2017-11-25 11:34:33,right
2285622,out,2017-11-26 16:53:17,right


### Collect in and out times as indexes and time and palce them in a dictionary

In [28]:
# in and out are times, and inx and outx are the indexes
in_out_dict = {"left":{"in":[],"out":[],"inx":[],"outx":[]},"middle":{"in":[],"out":[],"inx":[],"outx":[]},
               "right":{"in":[],"out":[],"inx":[],"outx":[]}}

for t in tnrange(len(rsv)):
    in_out_dict[rsv["Position"].iloc[t]][rsv["in_out"].iloc[t]].append(rsv["time"].iloc[t])
    in_out_dict[rsv["Position"].iloc[t]][rsv["in_out"].iloc[t]+'x'].append(rsv.index[t])
    
    

<IPython.core.display.Javascript object>

## Time Threshold
We now have a dictionary of all activity in our 3 urinal areas, timestamped.  
However, we still need to differentiate a person walking past the urinals (to take care of other business),   
and potentially remove any instantaneous temperature fluctuations  
(but hopefully our background temperature comparison took care of that).  

We call a person visiting the urinal, to relieve some pressure, a __stay__.  
A stay is defined as a person visisting the urinal longer than 8 seconds <sup>1</sup>.

___
<sup>1</sup> <sub> This was experimentally determined, and later verified by [this Ig Noble prize winning paper](http://www.pnas.org/content/111/33/11932?tab=ds), and two [Grideye](http://www.scirp.org/jouRNAl/PaperInformation.aspx?PaperID=74726) [specific](http://ieeexplore.ieee.org/document/6798925/) papers </sub>


In [29]:
from datetime import timedelta

# removes all non stays and sort into order of occurence
def removeNonStays(timeFilt):
    positions = ["left","middle","right"]
    unsorted_stays = pd.DataFrame()
    for position in positions:
        curr_stays = pd.concat([pd.DataFrame(in_out_dict[position]["in"]),pd.DataFrame(in_out_dict[position]["out"]),
                           pd.DataFrame(in_out_dict[position]["inx"]),pd.DataFrame(in_out_dict[position]["outx"])],axis=1)       
        curr_stays.columns = ["in","out","in_index","out_index"]
        curr_stays = curr_stays[curr_stays["out"] - curr_stays["in"]> timedelta(seconds=timeFilt )]
        curr_stays["length"] = curr_stays["out"] - curr_stays["in"]
        curr_stays["Position"] = position
        unsorted_stays = unsorted_stays.append(curr_stays)
        
        sorted_stays = unsorted_stays.sort_index()
        
    return sorted_stays



### Time filter for 8 seconds

In [31]:
timeFilter = 8
sorted_stays = removeNonStays(timeFilter)

removeNonStays(timeFilter).head()

,in,out,in_index,out_index,length,Position
23,2017-11-23 17:05:18,2017-11-23 17:05:52,2509,2814,00:00:34,left
28,2017-11-23 17:06:30,2017-11-23 17:07:12,3148,3520,00:00:42,left
37,2017-11-23 17:03:22,2017-11-23 17:03:48,1487,1719,00:00:26,right
44,2017-11-23 17:04:28,2017-11-23 17:05:15,2073,2486,00:00:47,right
69,2017-11-23 17:40:15,2017-11-23 17:40:52,21033,21364,00:00:37,left


In [32]:
pickle_full = "sorted_stays.p"

In [33]:
if os.path.isfile(pickle_full):
    print("You've already pickled!")
   
else:
    sorted_stays.to_pickle(pickle_full)

You've already pickled!


Go to __3 - Data Analysis__